In [1]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

c:\Users\ronny\AppData\Local\Programs\Python\Python312\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:

pc.create_index(
    name="rag-new",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '59e80d0f455ca082687de7bc51195776', 'Date': 'Mon, 26 Aug 2024 05:42:44 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [2]:
# Load the review data
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Great professor! Very clear lectures and helpful during office hours.'},
 {'professor': 'Dr. Alice Johnson',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Good explanations, but sometimes a bit fast-paced.'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Decent lectures, but not very engaging.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Material is hard, and lectures are confusing.'},
 {'professor': 'Dr. Linda Miller',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Very knowledgeable and approachable.'},
 {'professor': 'Dr. Richard Wilson',
  'subject': 'History',
  'stars': 5,
  'review': 'Passionate about the subject and makes history interesting.'},
 {'professor': 'Dr. Susan Lee',
  'subject': 'Literature',
  'stars': 4,
  'review': 'Engaging lectures, but the reading load is heavy.'},
 {'prof

In [3]:
processed_data = []
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )


KeyboardInterrupt: 

In [4]:

# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

# Print index statistics
print(index.describe_index_stats())

Upserted count: 3
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 3}},
 'total_vector_count': 3}
